In [1]:
answers = {"hosts": ["amy poehler","tina fey"],
"award_data": {
"best screenplay - motion picture": {
"nominees": [
"the grand budapest hotel",
"gone girl",
"boyhood",
"the imitation game"
],
"presenters": [
"bill hader",
"kristen wiig"
],
"winner": "birdman"
},
"best director - motion picture": {
"nominees": [
"wes anderson",
"ava duvernay",
"david fincher",
"alejandro inarritu gonzalez"
],
"presenters": [
"harrison ford"
],
"winner": "richard linklater"
},
"best performance by an actress in a television series - comedy or musical": {
"nominees": [
"lena dunham",
"edie falco",
"julia louis-dreyfus",
"taylor schilling"
],
"presenters": [
"bryan cranston",
"kerry washington"
],
"winner": "gina rodriguez"
},
"best foreign language film": {
"nominees": [
"force majeure",
"gett: the trial of viviane amsalem",
"ida",
"tangerines"
],
"presenters": [
"colin farrell",
"lupita nyong'o"
],
"winner": "leviathan"
},
"best performance by an actor in a supporting role in a motion picture": {
"nominees": [
"robert duvall",
"edward norton",
"mark ruffalo"
],
"presenters": [
"jennifer aniston",
"benedict cumberbatch"
],
"winner": "j.k. simmons"
},
"best performance by an actress in a supporting role in a series, mini-series or motion picture made for television": {
"nominees": [
"uzo aduba",
"kathy bates",
"allison janney",
"michelle monaghan"
],
"presenters": [
"jamie dornan",
"dakota johnson"
],
"winner": "joanne froggatt"
},
"best motion picture - comedy or musical": {
"nominees": [
"birdman",
"into the woods",
"pride",
"st. vincent"
],
"presenters": [
"robert downey, jr."
],
"winner": "the grand budapest hotel"
},
"best performance by an actress in a motion picture - comedy or musical": {
"nominees": [
"emily blunt",
"helen mirren",
"julianne moore",
"quvenzhane wallis"
],
"presenters": [
"ricky gervais"
],
"winner": "amy adams"
},
"best mini-series or motion picture made for television": {
"nominees": [
"the missing",
"the normal heart",
"olive kitteridge",
"true detective"
],
"presenters": [
"jennifer lopez",
"jeremy renner"
],
"winner": "fargo"
},
"best original score - motion picture": {
"nominees": [
"the imitation game",
"birdman",
"gone girl",
"interstellar"
],
"presenters": [
"sienna miller",
"vince vaughn"
],
"winner": "the theory of everything"
},
"best performance by an actress in a television series - drama": {
"nominees": [
"claire danes",
"viola davis",
"julianna margulies",
"robin wright"
],
"presenters": [
"anna faris",
"chris pratt"
],
"winner": "ruth wilson"
},
"best performance by an actress in a motion picture - drama": {
"nominees": [
"jennifer aniston",
"felicity jones",
"rosamund pike",
"reese witherspoon"
],
"presenters": [
"matthew mcconaughey"
],
"winner": "julianne moore"
},
"cecil b. demille award": {
"nominees": [],
"presenters": [
"don cheadle",
"julianna margulies"
],
"winner": "george clooney"
},
"best performance by an actor in a motion picture - comedy or musical": {
"nominees": [
"ralph fiennes",
"bill murray",
"joaquin phoenix",
"christoph waltz"
],
"presenters": [
"amy adams"
],
"winner": "michael keaton"
},
"best motion picture - drama": {
"nominees": [
"foxcatcher",
"the imitation game",
"selma",
"the theory of everything"
],
"presenters": [
"meryl streep"
],
"winner": "boyhood"
},
"best performance by an actor in a supporting role in a series, mini-series or motion picture made for television": {
"nominees": [
"alan cumming",
"colin hanks",
"bill murray",
"jon voight"
],
"presenters": [
"katie holmes",
"seth meyers"
],
"winner": "matt bomer"
},
"best performance by an actress in a supporting role in a motion picture": {
"nominees": [
"jessica chastain",
"keira knightley",
"emma stone",
"meryl streep"
],
"presenters": [
"jared leto"
],
"winner": "patricia arquette"
},
"best television series - drama": {
"nominees": [
"downton abbey (masterpiece)",
"game of thrones",
"the good wife",
"house of cards"
],
"presenters": [
"adam levine",
"paul rudd"
],
"winner": "the affair"
},
"best performance by an actor in a mini-series or motion picture made for television": {
"nominees": [
"martin freeman",
"woody harrelson",
"matthew mcconaughey",
"mark ruffalo"
],
"presenters": [
"jennifer lopez",
"jeremy renner"
],
"winner": "billy bob thornton"
},
"best performance by an actress in a mini-series or motion picture made for television": {
"nominees": [
"jessica lange",
"frances mcdormand",
"frances o'connor",
"allison tolman"
],
"presenters": [
"kate beckinsale",
"adrien brody"
],
"winner": "maggie gyllenhaal"
},
"best animated feature film": {
"nominees": [
"big hero 6",
"the book of life",
"the boxtrolls",
"the lego movie"
],
"presenters": [
"kevin hart",
"salma hayek"
],
"winner": "how to train your dragon 2"
},
"best original song - motion picture": {
"nominees": [
"big eyes",
"noah",
"annie",
"the hunger games: mockingjay - part 1"
],
"presenters": [
"prince"
],
"winner": "selma"
},
"best performance by an actor in a motion picture - drama": {
"nominees": [
"steve carell",
"benedict cumberbatch",
"jake gyllenhaal",
"david oyelowo"
],
"presenters": [
"gwyneth paltrow"
],
"winner": "eddie redmayne"
},
"best television series - comedy or musical": {
"nominees": [
"girls",
"jane the virgin",
"orange is the new black",
"silicon valley"
],
"presenters": [
"bryan cranston",
"kerry washington"
],
"winner": "transparent"
},
"best performance by an actor in a television series - drama": {
"nominees": [
"clive owen",
"liev schreiber",
"james spader",
"dominic west"
],
"presenters": [
"david duchovny",
"katherine heigl"
],
"winner": "kevin spacey"
},
"best performance by an actor in a television series - comedy or musical": {
"nominees": [
"louis c.k.",
"don cheadle",
"ricky gervais",
"william h. macy"
],
"presenters": [
"jane fonda",
"lily tomlin"
],
"winner": "jeffrey tambor"
}
}
}

print(answers["award_data"].keys())

dict_keys(['best screenplay - motion picture', 'best director - motion picture', 'best performance by an actress in a television series - comedy or musical', 'best foreign language film', 'best performance by an actor in a supporting role in a motion picture', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best mini-series or motion picture made for television', 'best original score - motion picture', 'best performance by an actress in a television series - drama', 'best performance by an actress in a motion picture - drama', 'cecil b. demille award', 'best performance by an actor in a motion picture - comedy or musical', 'best motion picture - drama', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actress in

In [7]:
import spacy
import re
nlp = spacy.load('en_core_web_sm')


def find_persons(text):
    # Create Doc object
    doc2 = nlp(text)

    # Identify the persons
    persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

    # Return persons
    return persons

def find_films(text):
    # Create Doc object
    doc2 = nlp(text)

    # ID the films
    films = [ent.text for ent in doc2.ents if ent.label_ == 'WORK_OF_ART']

    return films

def hasStopWord(x):
    stopWords = set(['Also', 'Sunday', 'Happy', 'Show', 'Best', 'Current', 'Tv', 'Award', 'No', 'It', 'What', 'Are', 'To', 'Win', 'My', \
        'Will', 'On', 'This', 'So', 'Here', 'Go', 'Actors', 'And', 'Who', 'Have', 'These', 'Is', 'Did', 'Be', 'She', 'He', 'With', 'The', 'A'])
    for word in stopWords:
        if word in x:
            return True
    return False

def isHypothetical(text):
    if '?' in text or 'last year' in text or 'hope' in text or 'hoping' in text or 'bet' in text or \
        'think' in text or 'will' in text or 'predict' in text or 'going to' in text or 'gonna' in text or 'should' in text:
        return True
    return False

def isHistorical(text):
    if (re.findall(r"\d\d\d\d", text) and '2015' not in text and '2014' not in text) or 'last year' in text:
        return True
    return False

def isReasonable(text):
    if not isHypothetical(text.lower()) and not isHistorical(text.lower()):
        return True
    return False

def isWinningTweet(text, awardIndicators, trips = []):
    """
        <text> = tweet text
        <award> = list of indicators that belong to a given award
    """
    text = text.lower()
    winIndicators = ['won', 'win', 'congrat', 'goes to', 'went to', 'snag', 'takes home', 'took home']
    missingWordCount = 0
    for i in winIndicators: #we want an indication of win
        if i in text:
            for a in awardIndicators:
                if a not in text:
                    # return False # we want an indication of our relevant award
                    missingWordCount += 1
            # if float(missingWordCount) / float(len(awardIndicators)) > 0.2:
            if float(missingWordCount) > 0.5 * float(len(awardIndicators)) - 2: #no missing words allowed from 4-word award, 1 missing word allowed from 6-word award
                return False # not enough of our award indicator words in tweet
                #
            for t in trips:
                if t in text:
                    return False # we do not want indication of a tripword
            return True
    # does not contain an indicator of a winning tweet
    return False

def awardNameToKeywords(text):
    stops = ['by', 'an', 'a', 'or', 'in', 'for', '-']
    l = text.lower().split()
    for s in stops:
        for i in range(l.count(s)):
            l.remove(s)
    for i in range(len(l)):
        l[i] = l[i].replace(',', '')
    return l

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def listDif(lst1, lst2): #lst1 - lst2
    lst3 = [value for value in lst1 if value not in lst2]
    return lst3

def findTripwords(award, awardList):
    tripwords = set(())
    for b in awardList:
        # print("A", award.keywords)
        # print("B", b.keywords)
        # print(len(award.keywords))
        # print(len(b.keywords))
        # print("INT", len(intersection(award.keywords, b.keywords)))

        # print("sharescore:", float(len(intersection(award.keywords,b.keywords))) / float(len(award.keywords)))
        if award.keywords == b.keywords: # avoid comparing
            pass
        elif float(len(intersection(award.keywords,b.keywords))) / float(len(award.keywords)) > .5 or \
            float(len(intersection(award.keywords,b.keywords))) / float(len(b.keywords)) > .5: #sufficiently similar award names
        # else:
            for word in listDif(b.keywords, award.keywords): # pulls out words that are in other award names but not in our award of interest
                # if word not in tripwords:
                    # print("adding ", word)
                tripwords.add(word)
    return list(tripwords)

In [3]:
import json
import re
import csv

# Opening JSON file
f = open('gg2015.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

print("loaded file")

tt = []
# print first 100 tweets
for tweet in data:
    # print(tweet['text'], data[99]['user']['screen_name'])
    tt.append(tweet['text'])
del data

f.close()

loaded file


In [4]:
class award:
    def __init__(self, name = "", keywords = [], tripwords = []):
        self.name = name
        self.keywords = keywords
        self.tripwords = tripwords
        self.winner = ""

In [5]:
# create list of award objects
awardList = []
for a in answers["award_data"].keys():
    # print(a)
    # print(awardNameToKeywords(a))
    awardList.append(award(name = a, keywords = awardNameToKeywords(a)))

# find tripwords!
for a in awardList:
    a.tripwords = findTripwords(award = a, awardList = awardList)

for a in awardList:
    print(a.name)
    print(a.keywords)
    print(a.tripwords)

best screenplay - motion picture
['best', 'screenplay', 'motion', 'picture']
['drama', 'made', 'role', 'performance', 'series', 'original', 'supporting', 'musical', 'television', 'song', 'director', 'actress', 'score', 'comedy', 'mini-series', 'actor']
best director - motion picture
['best', 'director', 'motion', 'picture']
['drama', 'made', 'role', 'performance', 'series', 'original', 'supporting', 'musical', 'television', 'screenplay', 'song', 'actress', 'score', 'comedy', 'mini-series', 'actor']
best performance by an actress in a television series - comedy or musical
['best', 'performance', 'actress', 'television', 'series', 'comedy', 'musical']
['drama', 'made', 'role', 'picture', 'supporting', 'mini-series', 'motion', 'actor']
best foreign language film
['best', 'foreign', 'language', 'film']
[]
best performance by an actor in a supporting role in a motion picture
['best', 'performance', 'actor', 'supporting', 'role', 'motion', 'picture']
['drama', 'made', 'series', 'actress', 'o

In [8]:
# AWARD WINNERS:
# THIS BOX COMBINES NAMES AND WORKS - highly dynamic without hand-tuning, but is not as accurate as if we were able to divide awards by
# human vs title recipients.

ttr = [] # pruned tweets by reasonability - i.e. not hypothetical and not historic
for t in tt:
    if isReasonable(t) and 'RT' not in t:
        ttr.append(t)
# find winner of every award
for ggAward in awardList:
    # print(ggAward.name)
    winningTweets = []
    # iter over tweets and add tweets that could contain the answer to our question
    for t in ttr:
        if isWinningTweet(t, ggAward.keywords, ggAward.tripwords):
            winningTweets.append(t)
    # in the case that we've been too restrictive, loosen constraints - no tripwords
    if len(winningTweets) == 0:
        for t in ttr:
            if isWinningTweet(t, ggAward.keywords):
                winningTweets.append(t)

    print("----------------------------------------------------------------")
    print("number of winning tweets found for : ", ggAward.name, " : ", len(winningTweets))

    # candWinners is a dict of counts of co-occurance of each candidate for winning. in the end we return the most popular name from the tweets.
    candWinners = {}
    # set for hash table speed
    namesSet = set(())

    for t in winningTweets:
        people = find_persons(t)
        films = find_films(t)
        # print(t)
        # print(films)
        # print(people)
        for p in people:
            if '@' in p or 'RT' in p or 'golden' in p:
                people = people.remove(p)
        for f in films:
            if '@' in f or 'RT' in f or 'golden' in f.lower():
                films = films.remove(f)
        if not people and not films:
            continue
        # print(people)
        # find counts
        if people:
            for p in people:
                if p in namesSet:
                    candWinners[p] += 1
                else:
                    candWinners[p] = 1
                namesSet.add(p)
        if films:
            for f in films:
                if f in namesSet:
                    candWinners[f] += 1
                else:
                    candWinners[f] = 1
                namesSet.add(f)

    # post processing - cleaning
    toDelete = []
    for name in candWinners.keys():
        if '@' in name:
            # print("@ found")
            toDelete.append(name)
            continue
        elif 'golden' in name.lower():
            # print("golden found")
            toDelete.append(name)
            continue
        ns = name.lower().split()
        # print("ns", ns)
        nsl = 0
        for i in ns:
            # print(i, ' in ', ggAward.name, ' ? ')
            # print(i in ggAward.name)
            if i in ggAward.name:
                nsl += 1
        if nsl == len(ns):
            # print("mistaken award name for recipient")
            toDelete.append(name)

    for d in toDelete:
        candWinners.pop(d)
    
    # sort by popularity then print winner
    winnerCounts = (sorted(candWinners.items(), key=lambda item: 1/item[1]))
    print("predicted winner: ", winnerCounts[0][0])
    # print(winnerCounts)

----------------------------------------------------------------
number of winning tweets found for :  best screenplay - motion picture  :  137
predicted winner:  Birdman Wins
----------------------------------------------------------------
number of winning tweets found for :  best director - motion picture  :  216
predicted winner:  Richard Linklater
----------------------------------------------------------------
number of winning tweets found for :  best performance by an actress in a television series - comedy or musical  :  19
predicted winner:  Jane the Virgin
----------------------------------------------------------------
number of winning tweets found for :  best foreign language film  :  434
predicted winner:  Leviathan
----------------------------------------------------------------
number of winning tweets found for :  best performance by an actor in a supporting role in a motion picture  :  18
predicted winner:  J.K. Simmons
-----------------------------------------------

In [13]:
# THIS BOX IS MEANT TO FIND PEOPLE ONLY - works really well when we know that the award recipient is a person and not a movie/show
# awardList[1] = best director
winningTweets = []
# iter over tweets and add tweets that could contain the answer to our question
for t in tt:
    if isReasonable(t) and isWinningTweet(t, awardList[1].keywords, awardList[1].tripwords):
        winningTweets.append(t)

print("number of winning tweets found for : ", awardList[1].name, " = ", len(winningTweets))

candWinners = {}
namesSet = set(())

for t in winningTweets:
    people = find_persons(t)
    films = find_films(t)
    # print(t)
    # print(films)
    # print(people)
    for p in people:
        if '@' in p or 'RT' in p or 'golden' in p:
            people = people.remove(p)
    if not people:
        continue
    # print(people)
    for p in people:
        if p in namesSet:
            candWinners[p] += 1
        else:
            candWinners[p] = 1
        namesSet.add(p)
winnerCounts = (sorted(candWinners.items(), key=lambda item: 1/item[1]))
print(winnerCounts[0][0])

number of winning tweets found for :  best director - motion picture  =  982
Richard Linklater


In [ ]:
# create a sample award indicators list "best performance by an actor in a motion picture - comedy or musical": {
bestActor = ["actor", "comedy", "motion", "best", "musical"]
trips = ["supporting", "nom", "tv", "television", "drama"]
#if this needs to be automated: find the closest award names to a given award: i.e. (best actress in a motion picture drama)
# would be similar to (best actress in a motion picture comedy or musical) and (best actress in a tv movie) or whatever,
# then pick out the words that are in similar award names but not in the one of interest. those are our tripwords.

# iterate over tweets and add counts to those actors that won an award
winningTweets = []
for t in tt:
    if isReasonable(t) and isWinningTweet(t, bestActor, trips):
        winningTweets.append(t)

print("number of winning tweets found for : ", bestActor, " = ", len(winningTweets))

candWinners = {}
actorNamesSet = set(())
candAts = {}
atsSet = set(())

for t in winningTweets[:1000]:
    people = find_persons(t)
    # print(t)
    # print(people)
    for p in people:
        if '@' in p or 'RT' in p or 'golden' in p:
            people = people.remove(p)
    if not people:
        continue
    # print(people)
    ats = re.findall(r'@[a-zA-Z_\-\.]+', t)
    # if ats:
        # print(ats)
    for p in people:
        if p in actorNamesSet:
            candWinners[p] += 1
        else:
            candWinners[p] = 1
        actorNamesSet.add(p)
    for a in ats:
        if a in atsSet:
            candAts[a] += 1
        else:
            candAts[a] = 1
        atsSet.add(a)
winnerCounts = (sorted(candWinners.items(), key=lambda item: 1/item[1]))
winnerCountsAts = (sorted(candAts.items(), key=lambda item: 1/item[1]))
# print(winnerCounts[0][0])

number of winning tweets found for :  ['actor', 'comedy', 'motion', 'best', 'musical']  =  1487


In [7]:
print(winnerCounts[0][0])
print(winnerCounts)


Richard Linklater
[('Richard Linklater', 551), ('RICHARD LINKLATER', 17), ('Ellar Coltrane', 14), ('Winner', 10), ('Ethan Haw', 9), ('Motion Picture', 6), ('Congrats Richard Linklater', 3), ('Wes Anderson', 3), ('GoldenGlobes http://t.co/Wg3jU3EPsU”', 2), ('Ethan Hawke', 2), ('richard linklater', 1), ('GoldenGlobes http://t.co/7SZ7xsA9cZ', 1), ('Richard Linklater - Boyhood', 1), ('😴', 1), ('GoldenGlobes http://t.co/T1LMPyLHKy', 1), ('Sam Houston', 1), ('http://t.co/Dp1uQxEnvU', 1), ('Ethan', 1), ('Ellar Coltra', 1), ('- Congrats Richard', 1), ('David Fincher', 1), ('Golden Globes', 1), ('Congrats Wes', 1), ('Rick', 1)]


In [6]:
print(atsSet)
print(winnerCountsAts)
print(winnerCountsAts)


{'@NYDNEntertain', '@goldenglobes', '@THR', '@eonline', '@HuffPostEnt', '@IMDb', '@ChannelGuide', '@Vh', '@TheBlackIcarus', '@ABC', '@WhoWhatWear', '@GossipCop', '@CBSThisMorning', '@ForbesLife', '@MokinoID', '@coslive', '@HuffingtonPost', '@ReelCritic', '@nypost', '@CineplexMovies', '@dna', '@WeGeekGirls', '@GMA', '@CharterCom', '@B', '@MichaelKeaton', '@Nbc', '@HOLLYWIRE', '@latimesent', '@NerdistDotCom', '@mituvoice', '@Zap'}
[('@eonline', 186), ('@IMDb', 164), ('@HuffingtonPost', 57), ('@NerdistDotCom', 28), ('@nypost', 16), ('@WhoWhatWear', 13), ('@MichaelKeaton', 10), ('@Zap', 8), ('@CineplexMovies', 7), ('@ForbesLife', 7), ('@goldenglobes', 4), ('@GMA', 4), ('@THR', 4), ('@CBSThisMorning', 4), ('@GossipCop', 3), ('@ABC', 3), ('@HOLLYWIRE', 3), ('@mituvoice', 3), ('@latimesent', 2), ('@coslive', 2), ('@dna', 2), ('@TheBlackIcarus', 2), ('@NYDNEntertain', 1), ('@ChannelGuide', 1), ('@ReelCritic', 1), ('@MokinoID', 1), ('@Nbc', 1), ('@CharterCom', 1), ('@WeGeekGirls', 1), ('@HuffPo